# Needleman–Wunsch (Global Alignment) for Hemoglobin α Chains

This notebook implements the Needleman–Wunsch algorithm with configurable scoring matrices (BLOSUM62, BLOSUM80, PAM250) and affine gap penalties (default gap open 11, gap extend 1). It loads sequences from `hemoglobin_209_species_final.fasta` and metadata from `hemoglobin_209_species_final.csv`, supports pilot and full pairwise runs, and exports results for clustering.


In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.6 MB/s eta 0:00:00


In [ ]:
# Imports and configuration
from Bio import SeqIO
from Bio.Align import substitution_matrices
import pandas as pd
from dataclasses import dataclass
from typing import Dict, Tuple, List
import itertools
import math
import time
import numpy as np
from numba import njit

# Paths
FASTA_PATH = "hemoglobin_209_species_final_fasta.fasta"
CSV_PATH = "hemoglobin_209_species_final.csv"

# Default parameters
DEFAULT_MATRIX_NAME = "BLOSUM62"
DEFAULT_GAP_OPEN = 11
DEFAULT_GAP_EXTEND = 1

# Allowed matrix names (used with substitution_matrices.load)
VALID_MATRIX_NAMES = ["BLOSUM62", "BLOSUM80", "PAM250"]

@dataclass
class SequenceRecord:
    protein_id: str
    species: str
    order: str
    superorder: str
    sequence: str



In [ ]:
# Load sequences and metadata

def load_sequences(fasta_path: str, csv_path: str) -> List[SequenceRecord]:
    meta = pd.read_csv(csv_path)
    # Ensure index type matches FASTA ids (strings)
    meta['protein_id'] = meta['protein_id'].astype(str)
    meta = meta.set_index('protein_id')

    records: List[SequenceRecord] = []
    for rec in SeqIO.parse(fasta_path, 'fasta'):
        pid = str(rec.id)
        if pid not in meta.index:
            continue
        row = meta.loc[pid]
        records.append(
            SequenceRecord(
                protein_id=pid,
                species=str(row['species']),
                order=str(row['order']),
                superorder=str(row['superorder']),
                sequence=str(rec.seq).upper().replace('U', 'X')  # standardize
            )
        )
    return records

seq_records = load_sequences(FASTA_PATH, CSV_PATH)
print(f"Loaded sequences: {len(seq_records)}")


Loaded sequences: 209


In [ ]:
# Scoring matrix helpers (symmetric lookup)

def load_matrix(name: str) -> Dict[Tuple[str, str], int]:
    name = name.upper()
    if name not in VALID_MATRIX_NAMES:
        raise ValueError(f"Unsupported matrix '{name}'. Choose from {VALID_MATRIX_NAMES}")
    mat = substitution_matrices.load(name)
    alphabet = mat.alphabet
    result: Dict[Tuple[str, str], int] = {}
    for a in alphabet:
        for b in alphabet:
            score = int(mat[a, b])
            result[(a, b)] = score
            result[(b, a)] = score
    return result

# Example: mat = load_matrix(DEFAULT_MATRIX_NAME)



In [ ]:
# Needleman–Wunsch (global alignment) — affine gap (Gotoh)
NEG_INF = -10**9

# -------------------------------------------------------------------
# Amino acid setup (you can reuse your existing AMINO_ACIDS if you want)
# -------------------------------------------------------------------
AMINO_ACIDS = [
    'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
    'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V',
    'B', 'Z', 'J', 'X', 'U', 'O'
]

aa_to_idx = {aa: i for i, aa in enumerate(AMINO_ACIDS)}
N_AA = len(AMINO_ACIDS)

NEG_INF = -10**9
NEG_INF_F = -1e9  # float version for DP arrays


def build_subst_array(matrix_dict: Dict[Tuple[str, str], int],
                      gap_open: int) -> np.ndarray:
    """
    Convert your (a,b)->score dict into a NumPy array for Numba.

    Matches your original:
      score_sub = matrix.get((a, b), matrix.get((b, a), -gap_open))
    """
    arr = np.zeros((N_AA, N_AA), dtype=np.int16)
    default_val = -gap_open
    for i, a in enumerate(AMINO_ACIDS):
        for j, b in enumerate(AMINO_ACIDS):
            arr[i, j] = matrix_dict.get(
                (a, b),
                matrix_dict.get((b, a), default_val)
            )
    return arr.astype(np.float32)


def encode_seq(seq: str) -> np.ndarray:
    """Encode amino acid sequence to integer indices."""
    seq = seq.upper()
    enc = np.zeros(len(seq), dtype=np.int16)
    for i, aa in enumerate(seq):
        enc[i] = aa_to_idx.get(aa, 0)
    return enc


@njit
def nw_affine_fill(seq1_enc,
                   seq2_enc,
                   subst_array,
                   gap_open,
                   gap_extend):
    """
    Fill M, Ix, Iy matrices for global Needleman–Wunsch with affine gaps.
    """
    n = len(seq1_enc)
    m = len(seq2_enc)

    M = np.full((n + 1, m + 1), NEG_INF_F, dtype=np.float32)
    Ix = np.full((n + 1, m + 1), NEG_INF_F, dtype=np.float32)
    Iy = np.full((n + 1, m + 1), NEG_INF_F, dtype=np.float32)

    # Initialization (global alignment, affine gaps)
    M[0, 0] = 0.0

    # First column: gaps in seq2 (Ix)
    for i in range(1, n + 1):
        Ix[i, 0] = -gap_open - (i - 1) * gap_extend

    # First row: gaps in seq1 (Iy)
    for j in range(1, m + 1):
        Iy[0, j] = -gap_open - (j - 1) * gap_extend

    # DP recurrence
    for i in range(1, n + 1):
        a_idx = seq1_enc[i - 1]
        for j in range(1, m + 1):
            b_idx = seq2_enc[j - 1]
            score_sub = subst_array[a_idx, b_idx]

            # M state
            best_prev = M[i - 1, j - 1]
            if Ix[i - 1, j - 1] > best_prev:
                best_prev = Ix[i - 1, j - 1]
            if Iy[i - 1, j - 1] > best_prev:
                best_prev = Iy[i - 1, j - 1]
            M[i, j] = best_prev + score_sub

            # Ix state (gap in seq2)
            best_ix = M[i - 1, j] - gap_open
            tmp = Ix[i - 1, j] - gap_extend
            if tmp > best_ix:
                best_ix = tmp
            Ix[i, j] = best_ix

            # Iy state (gap in seq1)
            best_iy = M[i, j - 1] - gap_open
            tmp = Iy[i, j - 1] - gap_extend
            if tmp > best_iy:
                best_iy = tmp
            Iy[i, j] = best_iy

    return M, Ix, Iy


def nw_global_affine(seq1: str,
                     seq2: str,
                     matrix: Dict[Tuple[str, str], int],
                     gap_open: int,
                     gap_extend: int):
    """
    Needleman–Wunsch (global alignment) — affine gap (Gotoh),
    accelerated with NumPy + Numba.

    Same interface and output keys as your original code1.
    """
    # Encode sequences
    seq1 = seq1.upper()
    seq2 = seq2.upper()
    seq1_enc = encode_seq(seq1)
    seq2_enc = encode_seq(seq2)

    # Build substitution matrix array
    # (you can cache this globally later if you want)
    subst_array = build_subst_array(matrix, gap_open)

    # Run Numba-accelerated DP
    M, Ix, Iy = nw_affine_fill(
        seq1_enc,
        seq2_enc,
        subst_array,
        float(gap_open),
        float(gap_extend)
    )

    n = len(seq1)
    m = len(seq2)

    # Choose best ending state at (n, m)
    end_scores = [M[n, m], Ix[n, m], Iy[n, m]]
    end_states = ['M', 'Ix', 'Iy']
    best_idx = int(np.argmax(np.array(end_scores)))
    score_final = float(end_scores[best_idx])
    state = end_states[best_idx]

    # Traceback (Python, but uses the numeric matrices)
    aln1 = []
    aln2 = []
    matches = 0
    # Removed 'positives' and 'aligned_non_gap' as they are recalculated
    # in the new metric block to ensure consistency with the other algorithms

    tol = 1e-6  # float comparison tolerance
    i, j = n, m

    while i > 0 or j > 0:
        # Handle edges
        if i > 0 and j == 0:
            aln1.append(seq1[i - 1])
            aln2.append('-')
            i -= 1
            continue
        elif j > 0 and i == 0:
            aln1.append('-')
            aln2.append(seq2[j - 1])
            j -= 1
            continue

        if state == 'M':
            a = seq1[i - 1]
            b = seq2[j - 1]
            a_idx = seq1_enc[i - 1]
            b_idx = seq2_enc[j - 1]
            score_sub = subst_array[a_idx, b_idx]

            aln1.append(a)
            aln2.append(b)

            # match/positive counting moved to post-processing for unified logic

            prev_M = M[i - 1, j - 1] + score_sub
            prev_Ix = Ix[i - 1, j - 1] + score_sub
            prev_Iy = Iy[i - 1, j - 1] + score_sub

            current_val = M[i, j]
            # decide source state
            if abs(current_val - prev_M) <= tol:
                state = 'M'
            elif abs(current_val - prev_Ix) <= tol:
                state = 'Ix'
            else:
                state = 'Iy'

            i -= 1
            j -= 1

        elif state == 'Ix':
            # gap in seq2
            aln1.append(seq1[i - 1])
            aln2.append('-')

            current_val = Ix[i, j]
            prev_from_M = M[i - 1, j] - gap_open
            prev_from_Ix = Ix[i - 1, j] - gap_extend

            if abs(current_val - prev_from_M) <= tol:
                state = 'M'
            else:
                state = 'Ix'

            i -= 1

        else:  # 'Iy'
            aln1.append('-')
            aln2.append(seq2[j - 1])

            current_val = Iy[i, j]
            prev_from_M = M[i, j - 1] - gap_open
            prev_from_Iy = Iy[i, j - 1] - gap_extend

            if abs(current_val - prev_from_M) <= tol:
                state = 'M'
            else:
                state = 'Iy'

            j -= 1

    aln1 = ''.join(reversed(aln1))
    aln2 = ''.join(reversed(aln2))

    # ---------------------------------------------------------
    # NEW METRIC LOGIC START
    # ---------------------------------------------------------

    matches = 0
    mismatches = 0
    gaps_seq1 = 0
    gaps_seq2 = 0

    for k in range(len(aln1)):
        if aln1[k] == '-':
            gaps_seq1 += 1
        elif aln2[k] == '-':
            gaps_seq2 += 1
        elif aln1[k] == aln2[k]:
            matches += 1
        else:
            mismatches += 1

    # Calculate Normalization Factor (Min Original Length)
    # This prevents Local Alignment from getting artificially low distances.
    # For Global, min_original_len is roughly the sequence length anyway.
    min_original_len = min(n, m)

    aligned_length = len(aln1)
    aligned_positions = matches + mismatches

    if min_original_len > 0:
        # Identity relative to the SHORTER sequence
        identity = matches / min_original_len
        distance = 1.0 - identity
    else:
        identity = 0.0
        distance = 1.0

    # Sanity check
    distance = max(0.0, distance)


    return {
        'score': score_final,
        'aln1': aln1,
        'aln2': aln2,
        'identity': identity,
        'distance': distance,
        'alignment_length': aligned_length,
        'matches': matches,
        'mismatches': mismatches,
        'gaps_seq1': gaps_seq1,
        'gaps_seq2': gaps_seq2
    }



In [ ]:
# Pilot run on a small subset (Just for 20 samples)
import random

random.seed(42)

matrix = load_matrix(DEFAULT_MATRIX_NAME)

def run_pair(a: SequenceRecord, b: SequenceRecord):
    res = nw_global_affine(a.sequence, b.sequence, matrix=matrix, gap_open=DEFAULT_GAP_OPEN, gap_extend=DEFAULT_GAP_EXTEND)
    return {
        'protein_id_1': a.protein_id,
        'protein_id_2': b.protein_id,
        'species_1': a.species,
        'species_2': b.species,
        'order_1': a.order,
        'order_2': b.order,
        'superorder_1': a.superorder,
        'superorder_2': b.superorder,
        'matrix': DEFAULT_MATRIX_NAME,
        'gap_open': DEFAULT_GAP_OPEN,
        'gap_extend': DEFAULT_GAP_EXTEND,
        **res,
    }

pilot_n = 20
subset = random.sample(seq_records, min(pilot_n, len(seq_records)))
pairs = list(itertools.combinations(subset, 2))

pilot_results = [run_pair(a, b) for a, b in pairs]

pilot_df = pd.DataFrame(pilot_results)
print(f"Pilot pairs: {len(pairs)}; Pilot rows: {len(pilot_df)}")
print("Columns:", list(pilot_df.columns))

if not pilot_df.empty and all(c in pilot_df.columns for c in ['score','identity','similarity']):
    pilot_df[['score','identity','similarity']].describe()
else:
    print("Pilot DataFrame is empty or missing expected columns ('score','identity','similarity').")


Pilot pairs: 190; Pilot rows: 190
Columns: ['protein_id_1', 'protein_id_2', 'species_1', 'species_2', 'order_1', 'order_2', 'superorder_1', 'superorder_2', 'matrix', 'gap_open', 'gap_extend', 'score', 'aln1', 'aln2', 'identity', 'distance', 'alignment_length', 'matches', 'mismatches', 'gaps_seq1', 'gaps_seq2']
Pilot DataFrame is empty or missing expected columns ('score','identity','similarity').


In [ ]:
# Pilot summary stats
if not pilot_df.empty:
    display(pilot_df[['score','identity','distance']].describe())
else:
    print("No pilot results to summarize.")


,score,identity,distance
count,190.000000,190.000000,190.000000
mean,361.784211,0.590906,0.409094
std,297.998350,0.282576,0.282576
min,-70.000000,0.147059,0.007092
25%,-42.750000,0.227941,0.197183
50%,552.500000,0.760563,0.239437
75%,592.000000,0.802817,0.772059
max,730.000000,0.992908,0.852941


In [ ]:
# Full pairwise run (may take time) (For all 209 samples)
from itertools import combinations

full_pairs = list(combinations(seq_records, 2))
print(f"Total pairs: {len(full_pairs)}")

mat = load_matrix(DEFAULT_MATRIX_NAME)

def run_pair_row(a: SequenceRecord, b: SequenceRecord):
    res = nw_global_affine(a.sequence, b.sequence, matrix=mat, gap_open=DEFAULT_GAP_OPEN, gap_extend=DEFAULT_GAP_EXTEND)
    return {
        'protein_id_1': a.protein_id,
        'protein_id_2': b.protein_id,
        'species_1': a.species,
        'species_2': b.species,
        'order_1': a.order,
        'order_2': b.order,
        'superorder_1': a.superorder,
        'superorder_2': b.superorder,
        'matrix': DEFAULT_MATRIX_NAME,
        'gap_open': DEFAULT_GAP_OPEN,
        'gap_extend': DEFAULT_GAP_EXTEND,
        **res,
    }

start = time.perf_counter()
full_results = [run_pair_row(a, b) for a, b in full_pairs]
elapsed = time.perf_counter() - start
print(f"Time to compute all alignments: {elapsed:.2f} seconds")

full_df = pd.DataFrame(full_results)
print(f"Full rows: {len(full_df)}")

# Save full results
full_df.to_csv('nw_full_results.csv', index=False)
print("Saved nw_full_results.csv")

# Build identity distance matrix: 1 - identity
ids = sorted({r.protein_id for r in seq_records})
idx = pd.Index(ids, name='protein_id')
dist_df = pd.DataFrame(1.0, index=idx, columns=ids)

for _, row in full_df[['protein_id_1','protein_id_2','identity']].iterrows():
    i = row['protein_id_1']
    j = row['protein_id_2']
    d = 1.0 - float(row['identity'])
    dist_df.at[i, j] = d
    dist_df.at[j, i] = d

# Zero diagonal
for i in ids:
    dist_df.at[i, i] = 0.0

# Save distance matrix
dist_df.to_csv('nw_distance_identity.csv')
print("Saved nw_distance_identity.csv")


Total pairs: 21736
Time to compute all alignments: 18.45 seconds
Full rows: 21736
Saved nw_full_results.csv
Saved nw_distance_identity.csv


In [ ]:
#Run full NW for multiple matrices (BLOSUM62, BLOSUM80, PAM250)
from itertools import combinations
import os

MATRICES = ["BLOSUM62", "BLOSUM80", "PAM250"]

# Precompute all pairs once
all_pairs = list(combinations(seq_records, 2))
print(f"Pairs to evaluate: {len(all_pairs)}")

def run_full_for_matrix(matrix_name: str):
    print(f"\n=== Running NW with {matrix_name} ===")
    mat = load_matrix(matrix_name)

    def run_pair_row(a: SequenceRecord, b: SequenceRecord):
        res = nw_global_affine(a.sequence, b.sequence, matrix=mat, gap_open=DEFAULT_GAP_OPEN, gap_extend=DEFAULT_GAP_EXTEND)
        return {
            'protein_id_1': a.protein_id,
            'protein_id_2': b.protein_id,
            'species_1': a.species,
            'species_2': b.species,
            'order_1': a.order,
            'order_2': b.order,
            'superorder_1': a.superorder,
            'superorder_2': b.superorder,
            'matrix': matrix_name,
            'gap_open': DEFAULT_GAP_OPEN,
            'gap_extend': DEFAULT_GAP_EXTEND,
            **res,
        }

    results = [run_pair_row(a, b) for a, b in all_pairs]
    df = pd.DataFrame(results)
    print(f"Rows: {len(df)}")

    # Save per-matrix results and distance matrix
    res_path = f"nw_full_results_{matrix_name}.csv"
    dist_path = f"nw_distance_identity_{matrix_name}.csv"
    df.to_csv(res_path, index=False)
    print(f"Saved {res_path}")

    ids = sorted({r.protein_id for r in seq_records})
    idx = pd.Index(ids, name='protein_id')
    dist_df = pd.DataFrame(1.0, index=idx, columns=ids)

    for _, row in df[['protein_id_1','protein_id_2','identity']].iterrows():
        i = row['protein_id_1']
        j = row['protein_id_2']
        d = 1.0 - float(row['identity'])
        dist_df.at[i, j] = d
        dist_df.at[j, i] = d

    for i in ids:
        dist_df.at[i, i] = 0.0

    dist_df.to_csv(dist_path)
    print(f"Saved {dist_path}")

# Run for BLOSUM80 and PAM250 (BLOSUM62 already done, but safe to re-run if desired)
for name in ["BLOSUM80", "PAM250"]:
    run_full_for_matrix(name)



Pairs to evaluate: 21736

=== Running NW with BLOSUM80 ===
Rows: 21736
Saved nw_full_results_BLOSUM80.csv
Saved nw_distance_identity_BLOSUM80.csv

=== Running NW with PAM250 ===
Rows: 21736
Saved nw_full_results_PAM250.csv
Saved nw_distance_identity_PAM250.csv


In [ ]:
#Compare matrices: within vs between superorders and matrix correlations
import numpy as np

# Load per-matrix distance matrices
mat_to_dist = {}
for name in ["BLOSUM62", "BLOSUM80", "PAM250"]:
    path = f"nw_distance_identity_{name}.csv" if name != "BLOSUM62" else "nw_distance_identity.csv"
    mat_to_dist[name] = pd.read_csv(path, index_col=0)

# Helper: compute within/between superorder means
pid_to_super = {str(r.protein_id): r.superorder for r in seq_records}

summary_rows = []
for name, dist_df in mat_to_dist.items():
    ids = list(dist_df.index)
    same, diff = [], []
    for i_idx in range(len(ids)):
        for j_idx in range(i_idx + 1, len(ids)):
            i = ids[i_idx]
            j = ids[j_idx]
            dij = float(dist_df.iloc[i_idx, j_idx])

            super_i = pid_to_super.get(str(i))
            super_j = pid_to_super.get(str(j))

            # skip pairs where we don't know superorder for either protein
            if super_i is None or super_j is None:
                continue

            if super_i == super_j:
                same.append(dij)
            else:
                diff.append(dij)

    summary_rows.append({
        'matrix': name,
      'mean_within_superorder': np.mean(same) if same else np.nan,
        'mean_between_superorder': np.mean(diff) if diff else np.nan,
        'separation_gap': (np.mean(diff) - np.mean(same)) if same and diff else np.nan,
    })

summary_df = pd.DataFrame(summary_rows).sort_values('separation_gap', ascending=False)
print("\nWithin/between superorder distance summary (higher gap is better):")
display(summary_df)

# Correlate distance matrices (upper-triangle Spearman)
from scipy.stats import spearmanr

def upper_triangle_values(df: pd.DataFrame) -> np.ndarray:
    vals = []
    n = df.shape[0]
    for i in range(n):
        for j in range(i + 1, n):
            vals.append(float(df.iloc[i, j]))
    return np.array(vals)

names = list(mat_to_dist.keys())
print("\nMatrix correlations (Spearman, upper triangle):")
for i in range(len(names)):
    for j in range(i + 1, len(names)):
        a = upper_triangle_values(mat_to_dist[names[i]])
        b = upper_triangle_values(mat_to_dist[names[j]])
        rho, p = spearmanr(a, b)
        print(f"{names[i]} vs {names[j]}: rho={rho:.3f}, p={p:.1e}")



Within/between superorder distance summary (higher gap is better):


,matrix,mean_within_superorder,mean_between_superorder,separation_gap
0,BLOSUM62,0.332270,0.355085,0.022815
1,BLOSUM80,0.298584,0.321261,0.022677
2,PAM250,0.340230,0.361272,0.021042



Matrix correlations (Spearman, upper triangle):
BLOSUM62 vs BLOSUM80: rho=0.984, p=0.0e+00
BLOSUM62 vs PAM250: rho=0.989, p=0.0e+00
BLOSUM80 vs PAM250: rho=0.982, p=0.0e+00


In [ ]:
# Validation: Compare NW implementation vs Biopython pairwise2
from Bio import pairwise2
from Bio.Align import substitution_matrices
import random

VALIDATION_MATRIX = DEFAULT_MATRIX_NAME  # e.g., 'BLOSUM62'
GAP_OPEN = DEFAULT_GAP_OPEN
GAP_EXTEND = DEFAULT_GAP_EXTEND

val_mat = substitution_matrices.load(VALIDATION_MATRIX)
our_val_matrix = load_matrix(VALIDATION_MATRIX)

# Sample pairs for validation
random.seed(7)
val_n = 15
val_subset = random.sample(seq_records, min(val_n, len(seq_records)))
val_pairs = list(itertools.combinations(val_subset, 2))

rows = []
for a, b in val_pairs:
    # Our NW (affine)
    our_res = nw_global_affine(a.sequence, b.sequence, matrix=our_val_matrix, gap_open=GAP_OPEN, gap_extend=GAP_EXTEND)

    # Biopython pairwise2 (use best alignment)
    alignments = pairwise2.align.globalds(a.sequence, b.sequence, val_mat, -GAP_OPEN, -GAP_EXTEND, one_alignment_only=True)
    if alignments:
        aln = alignments[0]
        bi_score = aln.score
        bi_aln1, bi_aln2 = aln.seqA, aln.seqB
        aligned_positions = sum(1 for x, y in zip(bi_aln1, bi_aln2) if x != '-' and y != '-')
        matches = sum(1 for x, y in zip(bi_aln1, bi_aln2) if x == y and x != '-' and y != '-')
        bi_identity = matches / aligned_positions if aligned_positions else 0.0
    else:
        bi_score = float('nan')
        bi_identity = float('nan')

    rows.append({
        'protein_id_1': a.protein_id,
        'protein_id_2': b.protein_id,
        'our_score': our_res['score'],
        'bi_score': bi_score,
        'our_identity': our_res['identity'],
        'bi_identity': bi_identity,
        'abs_diff_identity': abs(our_res['identity'] - bi_identity) if not pd.isna(bi_identity) else float('nan'),
        'abs_diff_score': abs(our_res['score'] - bi_score) if not pd.isna(bi_score) else float('nan'),
    })

val_df = pd.DataFrame(rows)
print(f"Validated pairs: {len(val_df)}")

print("\nIdentity comparison:")
display(val_df[['our_identity','bi_identity','abs_diff_identity']].describe())

print("\nScore comparison:")
display(val_df[['our_score','bi_score','abs_diff_score']].describe())

print("\nLargest identity differences:")
display(val_df.sort_values('abs_diff_identity', ascending=False).head(5))


/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Validated pairs: 105

Identity comparison:


,our_identity,bi_identity,abs_diff_identity
count,105.000000,105.000000,105.0
mean,0.676507,0.676507,0.0
std,0.196921,0.196921,0.0
min,0.166667,0.166667,0.0
25%,0.574468,0.574468,0.0
50%,0.758865,0.758865,0.0
75%,0.788732,0.788732,0.0
max,0.978873,0.978873,0.0



Score comparison:


,our_score,bi_score,abs_diff_score
count,105.000000,105.000000,105.0
mean,468.504762,468.504762,0.0
std,215.172119,215.172119,0.0
min,-63.000000,-63.000000,0.0
25%,417.000000,417.000000,0.0
50%,552.000000,552.000000,0.0
75%,586.000000,586.000000,0.0
max,724.000000,724.000000,0.0



Largest identity differences:


,protein_id_1,protein_id_2,our_score,bi_score,our_identity,bi_identity,abs_diff_identity,abs_diff_score
0,1430744588,15825904,562.0,562.0,0.765957,0.765957,0.0,0.0
1,1430744588,2591575512,576.0,576.0,0.774648,0.774648,0.0,0.0
2,1430744588,122495,623.0,623.0,0.836879,0.836879,0.0,0.0
3,1430744588,1561931297,543.0,543.0,0.718310,0.718310,0.0,0.0
4,1430744588,1586281612,593.0,593.0,0.788732,0.788732,0.0,0.0


In [ ]:
# Save pilot results
pilot_df.to_csv('nw_pilot_results.csv', index=False)
len(pilot_df), pilot_df.head(3)


(190,
   protein_id_1 protein_id_2           species_1              species_2  \
 0   1864417489   1934719391  Callithrix jacchus   Choloepus didactylus   
 1   1864417489     38502825  Callithrix jacchus  Monodelphis domestica   
 2   1864417489   2441695316  Callithrix jacchus     Nycticebus coucang   
 
     order_1          order_2      superorder_1      superorder_2    matrix  \
 0  Primates           Pilosa  Euarchontoglires         Xenarthra  BLOSUM62   
 1  Primates  Didelphimorphia  Euarchontoglires        Marsupials  BLOSUM62   
 2  Primates         Primates  Euarchontoglires  Euarchontoglires  BLOSUM62   
 
    gap_open  gap_extend  score  \
 0        11           1  -61.0   
 1        11           1  -51.0   
 2        11           1  390.0   
 
                                                 aln1  \
 0  MALLKANKDLISAGLKEFNVLLNQQV--FNDPLISE--EDMVIVVE...   
 1  MALLKANKDLISAGLKEF----NVLLNQQVFNDPLISEEDMVIVVE...   
 2  MALLKANKDLISAGLKEFNVLLNQQVFNDPLISEEDMVIVVEDWMN...   
 
  